In [7]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
trainDf = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submit = pd.read_csv('./submit_example.csv')

In [3]:
trainDf.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1.490000e+18,NaN,runail,2.62,1,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1.490000e+18,NaN,runail,2.62,1,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5723490,1.490000e+18,NaN,runail,2.62,1,26dd6e6e-4dac-4778-8d2c-92e149dab885
3,2019-10-01 00:02:32 UTC,cart,5857283,1.490000e+18,NaN,runail,2.62,1,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:02:40 UTC,cart,5723523,1.490000e+18,NaN,runail,2.62,1,26dd6e6e-4dac-4778-8d2c-92e149dab885


In [44]:
def check(df,user_id=1):
    gr = df.groupby('user_id')
    for i,ur in gr:
        if i!=user_id:
            continue
        #print(ur[['event_time','event_type','product_id','category_id','price','user_id']])
        print(np.unique(ur[['product_id']],return_counts=True))
        break
check(trainDf,3)
check(test,3)

(array([5723491, 5723518, 5773173, 5773216, 5773351, 5773379, 5814828,
       5814869, 5823288, 5857269, 5857343]), array([1, 2, 3, 1, 3, 1, 1, 4, 1, 2, 1]))


In [48]:
np.unique(trainDf['product_id'].to_numpy()).shape

(32734,)

In [49]:
trainDf['product_id'].describe()

count    4.169620e+05
mean     5.498023e+06
std      1.268676e+06
min      3.752000e+03
25%      5.729011e+06
50%      5.809809e+06
75%      5.850305e+06
max      5.892800e+06
Name: product_id, dtype: float64

In [46]:
test['product_id'].describe()

count    5.164000e+03
mean     5.452999e+06
std      1.342040e+06
min      3.762000e+03
25%      5.711138e+06
50%      5.799410e+06
75%      5.848896e+06
max      5.892678e+06
Name: product_id, dtype: float64

In [50]:
np.unique(test['product_id'].to_numpy()).shape

(2691,)

In [51]:
b=set(test['product_id'].to_numpy())
b.issubset(trainDf['product_id'].to_numpy())

False

In [53]:
for i in test['product_id'].to_numpy():
    if not np.isin(i,trainDf['product_id']):
        print(i)

5813942
5873068
5837436
5763624
5815681
5815681
5815681
5845219
5854967
5845220
5845220
5845219
5845219
5837803
5837803
5847229
5847229
5873063
5868869
5674716
5829781
5822733
5877549
5878911
5805908
5540773
5766066
5765739
5859333
5800947
5816633
5671067
5822497
5885135
5885135
5885133
5885133
5691629
5889160
5835137
5857894
5857894
5857895
5736285
5821166
5770528
5770528
5854118
5711027
5798780
5838089


KeyboardInterrupt: 

In [35]:
gr = trainDf.groupby(['user_id','product_id'])
for i,ur in gr:
    print(i)
    print(ur[['event_time','event_type','category_id','price','user_id']])
    if i[0]>1:
        break

(1, 5385)
                 event_time event_type   category_id  price  user_id
26  2019-10-01 00:12:50 UTC   purchase  1.490000e+18   3.89        1
(1, 5659639)
                event_time event_type   category_id  price  user_id
8  2019-10-01 00:12:50 UTC   purchase  1.490000e+18   0.95        1
(1, 5659825)
                event_time        event_type   category_id  price  user_id
6  2019-10-01 00:05:27 UTC              cart  1.490000e+18   0.95        1
7  2019-10-01 00:08:08 UTC  remove_from_cart  1.490000e+18   0.95        1
(1, 5688124)
                 event_time event_type   category_id  price  user_id
25  2019-10-01 00:12:50 UTC   purchase  1.490000e+18   0.32        1
(1, 5700046)
                 event_time event_type   category_id  price  user_id
24  2019-10-01 00:12:50 UTC   purchase  1.490000e+18    0.4        1
(1, 5723490)
                 event_time event_type   category_id  price  user_id
2   2019-10-01 00:00:07 UTC       cart  1.490000e+18   2.62        1
23  2019-10-

In [57]:
gr.size()

user_id  product_id
1        5385          1
         5659639       1
         5659825       2
         5688124       1
         5700046       1
                      ..
53972    5807884       2
         5877795       1
53973    5689159       1
53974    5809856       1
53975    5877385       1
Length: 213548, dtype: int64

In [23]:
print(trainDf.shape)
np.unique(trainDf['category_code'][trainDf['category_code'].notna()])

(416962, 9)


array(['accessories.bag', 'accessories.cosmetic_bag', 'apparel.glove',
       'appliances.environment.air_conditioner',
       'appliances.environment.vacuum', 'appliances.personal.hair_cutter',
       'furniture.bathroom.bath', 'furniture.living_room.cabinet',
       'furniture.living_room.chair', 'stationery.cartrige'], dtype=object)

In [29]:
np.unique(trainDf['category_id'][trainDf['category_id'].notna()])

array([1.49e+18, 1.50e+18, 1.51e+18, 1.52e+18, 1.53e+18, 1.54e+18,
       1.55e+18, 1.56e+18, 1.58e+18, 1.59e+18, 1.60e+18, 1.61e+18,
       1.63e+18, 1.64e+18, 1.65e+18, 1.66e+18, 1.72e+18, 1.73e+18,
       1.75e+18, 1.76e+18, 1.78e+18, 1.79e+18, 1.80e+18, 1.81e+18,
       1.82e+18, 1.84e+18, 1.86e+18, 1.87e+18, 1.89e+18, 1.90e+18,
       1.91e+18, 1.92e+18, 1.93e+18, 1.94e+18, 1.96e+18, 1.98e+18,
       1.99e+18, 2.00e+18, 2.01e+18, 2.02e+18, 2.03e+18, 2.04e+18,
       2.06e+18, 2.07e+18, 2.08e+18, 2.09e+18, 2.10e+18, 2.11e+18,
       2.12e+18, 2.13e+18, 2.14e+18, 2.15e+18, 2.16e+18, 2.19e+18,
       2.20e+18])

In [31]:
np.unique(trainDf['event_type'][trainDf['event_type'].notna()])

array(['cart', 'purchase', 'remove_from_cart', 'view'], dtype=object)

In [24]:
np.unique(trainDf['brand'][trainDf['brand'].notna()])

array(['airnails', 'almea', 'andrea', 'ardell', 'art-visage', 'artex',
       'australis', 'balbcare', 'barbie', 'batiste', 'beautix',
       'beauty-free', 'beautyblender', 'beauugreen', 'benovy', 'bergamo',
       'bespecial', 'binacil', 'bioaqua', 'biofollica', 'biore', 'blise',
       'blixz', 'bluesky', 'bodipure', 'bodyton', 'bosnic', 'bpw.style',
       'browxenna', 'busch', 'candy', 'carmex', 'chi', 'cnd', 'coifin',
       'concept', 'consly', 'coocla', 'cosima', 'cosmoprofi', 'coxir',
       'cruset', 'cuccio', 'cutrin', 'de.lux', 'deoproce', 'depilflax',
       'dermacol', 'dermal', 'dizao', 'domix', 'dorena', 'dr.gloderm',
       'egomania', 'elizavecca', 'ellips', 'elskin', 'embryolisse',
       'emil', 'enas', 'enigma', 'enjoy', 'entity', 'eos', 'estel',
       'estelare', 'eunyul', 'f.o.x', 'fancy', 'farmavita', 'farmona',
       'farmstay', 'fedua', 'finish', 'fly', 'foamie', 'freedecor',
       'freshbubble', 'frozen', 'gehwol', 'glysolid', 'godefroy', 'grace',
       '

In [19]:
def nonanRatio(df):
    entries = df.shape[0]
    for col in list(trainDf.columns):
        print('{}: {}/{} {:.5f}'.format(col, df[col][df[col].notna()].shape[0], entries, df[col][df[col].notna()].shape[0]/entries ))
nonanRatio(trainDf)

event_time: 416962/416962 1.00000
event_type: 416962/416962 1.00000
product_id: 416962/416962 1.00000
category_id: 416962/416962 1.00000
category_code: 6164/416962 0.01478
brand: 252018/416962 0.60441
price: 416962/416962 1.00000
user_id: 416962/416962 1.00000
user_session: 416924/416962 0.99991


In [21]:
nonanRatio(test)

event_time: 5164/5164 1.00000
event_type: 5164/5164 1.00000
product_id: 5164/5164 1.00000
category_id: 5164/5164 1.00000
category_code: 99/5164 0.01917
brand: 3088/5164 0.59799
price: 5164/5164 1.00000
user_id: 5164/5164 1.00000
user_session: 5162/5164 0.99961


In [11]:
(trainDf['category_id']/1e16).astype(int)

0         149
1         149
2         149
3         149
4         149
         ... 
416957    149
416958    149
416959    149
416960    178
416961    190
Name: category_id, Length: 416962, dtype: int64

In [23]:
train=trainDf[['event_time','event_type','product_id','category_id','price','user_id']]

In [27]:
train.loc[:,('category_id',)] = (trainDf['category_id']/1e16).astype(int)

In [41]:
#lbe = LabelEncoder()
#train.loc[:,('event_type')] = lbe.fit_transform(trainDf['event_type'])
train.loc[:,('event_type')] = trainDf['event_type'].apply(lambda x: eventmap[x])

/opt/gentoo/usr/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [40]:
event_type = ['cart', 'purchase', 'remove_from_cart', 'view']
eventmap = {'cart':5, 'purchase':10, 'remove_from_cart':-5, 'view':1}

In [42]:
def selectData(df):
    d=df[['event_time','event_type','product_id','category_id','price','user_id']]
    d.loc[:,('category_id',)] = (df['category_id']/1e16).astype(int)
    d.loc[:,('event_type')] = df['event_type'].apply(lambda x: eventmap[x])
    return d
test = selectData(test)

In [43]:
test.head()

,event_time,event_type,product_id,category_id,price,user_id
0,2019-10-03 08:14:04 UTC,1,5651977,149,31.73,53978
1,2019-10-03 08:14:41 UTC,1,37987,149,42.86,53978
2,2019-10-03 08:14:08 UTC,1,5753054,149,0.79,53980
3,2019-10-03 08:17:21 UTC,5,5753054,149,0.79,53980
4,2019-10-03 08:17:24 UTC,5,5753054,149,0.79,53980


In [46]:
test.to_hdf('test.h5','data')
train.to_hdf('train.h5','data')

In [2]:
test = pd.read_hdf('test.h5','data')
train = pd.read_hdf('train.h5','data')

In [3]:
def groupData(df):
    gr = df.groupby('user_id')
    matrix = gr.size().reset_index().rename(columns={0:'counts'})
    #print(matrix.head())
    temp = gr['product_id'].agg([('product_count','nunique')]).reset_index()
    #print(temp.head())
    matrix = matrix.merge(temp,on='user_id',how='left')
    #print(matrix.head())
    temp = gr['category_id'].agg([('category_count','nunique')]).reset_index()
    #print(temp.head())
    matrix = matrix.merge(temp,on='user_id',how='left')
    #print(matrix.head())
    label_list=[]
    for name, group in gr:
        product_id = int(group.iloc[-1,2])
        label_list.append([name,product_id])
    return matrix.merge(pd.DataFrame(label_list, columns=['user_id', 'label'], dtype=int), on='user_id', how='left')

In [4]:
testdata = groupData(test)

In [5]:
traindata = groupData(train)

In [6]:
x_train, y_train = traindata.drop(['label','user_id'], axis=1),traindata['label']

In [7]:
model =xgb.XGBClassifier(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300,
    colsample_bytree=0.8,
    subsample=0.8,
    eta=0.3,
    seed=42
)
model.fit(x_train,y_train,eval_metric='auc')

/opt/gentoo/usr/lib/python3.8/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


KeyboardInterrupt: 

In [ ]:
ytrain.head()

In [ ]:
test_data = test_data.drop(['user_id'], axis=1)
predict_test = model.predict(test_data)

In [ ]:
submit_example = pd.read_csv('./submit_example.csv')
submit_example['product_id'] = pd.Series(predict_test[:,0])
submit_example.to_csv('./xgb.csv',index=False)